In [ ]:
import pandas as pd
import re
from difflib import SequenceMatcher
import spacy
from spacy import displacy
zlp = spacy.load('zh_core_web_lg')
elp = spacy.load('en_core_web_lg')

In [8]:
import nltk
from nltk.grammar import CFG
from nltk.parse import RecursiveDescentParser

def get_root(doc):
    curr = doc[0]
    while curr.head != curr:
        curr = curr.head
    return curr


def to_nltk_tree(node):
    if len(list(node.children)) > 0:
        return nltk.Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

'''nltk_tree1 = to_nltk_tree(get_root(doc1))
nltk_tree2 = to_nltk_tree(get_root(doc2))
nltk_tree1.pretty_print()
nltk_tree2.pretty_print()'''

'nltk_tree1 = to_nltk_tree(get_root(doc1))\nnltk_tree2 = to_nltk_tree(get_root(doc2))\nnltk_tree1.pretty_print()\nnltk_tree2.pretty_print()'

In [56]:
class Node:
    def __init__(self, label, children=None):
        self.label = label
        self.children = children if children else []

    def get_children(node):
        return node.children

    def get_label(node):
        return node.label
    
    def complex_distance(token1, token2):
        return 0 if set(token1.children) == set(token2.children) else (len(token1.children))

def doc_to_tree(root):
    children = list(root.children)
    if len(children) == 0:
        return Node(root.dep_, [])
    else:
        return Node(root.dep_, [doc_to_tree(child) for child in children])

def pronoun_mask(tree):
    if tree.label != 'nsubj':
        tree.label = 0
    else:
        tree.label = 1
    children = tree.get_children()
    if len(children) > 0:
        for child in children: child = pronoun_mask(child)
    return tree

def collapse_pronouns(root):
    children = root.get_children()
    leaf_sum = 0
    branches = []
    num_childless = 0
    for child in children:
        if not child.get_children():
            leaf_sum += child.get_label()
        else:
            branches.append(collapse_pronouns(child))
    return Node(root.get_label(), [Node(leaf_sum)]+branches) if root.get_children() else Node(root.get_label())

def tree_to_set(root):

    def helper(root, prev, level):
        level = level
        elems = []
        children = root.get_children()
        for child in children:
            elems.append((level, prev, root.get_label(), child.get_label()))
            elems += helper(child, level, level + 1)
        return elems
    
    return [(0, -1, -1, root.get_label())] + helper(root, 0, 1)


In [55]:
doc1 = elp("When I came home I was tired")
dict = tree_to_dict(pronoun_mask(doc_to_tree(get_root(doc1))))

def dict_to_set(dictionary):
    s = set()
    for key in dictionary:
        nsubj = sum([tup[1] for tup in dictionary[key] if tup[2] == False])
        for tup in dictionary[key]:
            if tup[2] == False:
                dictionary[key].remove(tup)
                dictionary[key].append((tup[0], nsubj, tup[2]))
    
    for key in dictionary:
        for tup in dictionary[key]:
            print(tup)
            #s.add(dictionary[key])

dict_to_set(dict)


(0, 1, False)
(0, 1, False)
(0, 1, False)
(0, 1, False)
(0, 1, False)
(0, 1, False)


In [ ]:
def compare(doc1, doc2):
    set1 = set()
    set2 = set()
    ls1 = tree_to_dict(pronoun_mask(doc_to_tree(get_root(doc1))))
    ls2 = tree_to_dict(pronoun_mask(doc_to_tree(get_root(doc2))))
    for el in ls1:
        set1.add(el)
    for el in ls2:
        set2.add(el)
    return jaccard(set1, set2)

def jaccard(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
     
    return intersection / union if union else 0

In [57]:
df = pd.concat([pd.read_csv("./xinhua_scores.csv"), pd.read_csv("./xinhua_scores3.csv"), pd.read_csv("./xinhua_scores1.csv")], ignore_index=True)
df = df.drop_duplicates(subset=['zho'], keep='first')
df = df[df["zho"] != "\n"]
df = df[["zho", "eng", "zho_reverse"]]

similarities1 = []
similarities2 = []
zh_nsubjs = []
en_nsubjs = []
rev_nsubjs = []

for index, row in df.iterrows():
    doc1 = zlp(row["zho"])
    doc2 = elp(row["eng"])
    doc3 = zlp(row["zho_reverse"])
    similarity1 = compare(doc1, doc2)
    similarity2 = compare(doc2, doc3)
    similarities1.append(similarity1)
    similarities2.append(similarity2)
    zh_nsubjs.append(len([token.dep_ for token in doc1 if token.dep_ == 'nsubj']))
    en_nsubjs.append(len([token.dep_ for token in doc2 if token.dep_ == 'nsubj']))
    rev_nsubjs.append(len([token.dep_ for token in doc3 if token.dep_ == 'nsubj']))

df["zho_eng_jaccard"] = similarities1
df["eng_zho_jaccard"] = similarities2
df["zh_nsubjs"] = zh_nsubjs
df["en_nsubjs"] = en_nsubjs
df["rev_nsubjs"] = rev_nsubjs

In [58]:
df.to_csv("./final_xinhua_scores.csv", index = False)

In [ ]:
similarities1 = []
similarities2 = []
zh_nsubjs = []
en_nsubjs = []
rev_nsubjs = []

for index, row in df1.iterrows():
    doc1 = zlp(row["zho"])
    doc2 = elp(row["eng"])
    similarity = compare(doc1, doc2)
    similarities1.append(similarity)
    zh_nsubjs.append(len([token.dep_ for token in doc1 if token.dep_ == 'nsubj']))
    en_nsubjs.append(len([token.dep_ for token in doc2 if token.dep_ == 'nsubj']))

for index, row in df2.iterrows():
    doc1 = elp(row["eng"])
    doc2 = zlp(row["zho"])
    similarity = compare(doc1, doc2)
    similarities2.append(similarity)
    rev_nsubjs.append(len([token.dep_ for token in doc2 if token.dep_ == 'nsubj']))

df1["zho_reverse"] = df2["zho"]
df1["zho_eng_jaccard"] = similarities1
df1["eng_zho_jaccard"] = similarities2
df1["zh_nsubjs"] = zh_nsubjs
df1["en_nsubjs"] = en_nsubjs
df1["rev_nsubjs"] = rev_nsubjs

df1.to_csv("./xinhua_scores3.csv", index = False)

In [30]:
zho_eng_rev_1 = pd.read_csv('zho_eng_rev1.csv')

similarities1 = []
similarities2 = []
zh_nsubjs = []
en_nsubjs = []
rev_nsubjs = []

for index, row in zho_eng_rev_1.iterrows():
    doc1 = zlp(row["zho"])
    doc2 = elp(row["eng"])
    doc3 = zlp(row["zho_reverse"])
    similarity1 = compare(doc1, doc2)
    similarities1.append(similarity1)
    similarity2 = compare(doc2, doc3)
    similarities2.append(similarity2)
    zh_nsubjs.append(len([token.dep_ for token in doc1 if token.dep_ == 'nsubj']))
    en_nsubjs.append(len([token.dep_ for token in doc2 if token.dep_ == 'nsubj']))
    rev_nsubjs.append(len([token.dep_ for token in doc3 if token.dep_ == 'nsubj']))

zho_eng_rev_1["zho_reverse"] = df2["zho"]
zho_eng_rev_1["zho_eng_jaccard"] = similarities1
zho_eng_rev_1["eng_zho_jaccard"] = similarities2
zho_eng_rev_1["zh_nsubjs"] = zh_nsubjs
zho_eng_rev_1["en_nsubjs"] = en_nsubjs
zho_eng_rev_1["rev_nsubjs"] = rev_nsubjs

zho_eng_rev_1.to_csv('xinhua_scores1.csv', index = False)